In [1]:
class matchrow:
    def __init__(self,row,allnum=False):
        if allnum:
            self.data = [float(row[i]) for i in range(len(row)-1)]
        else:
            self.data = row[0:len(row)-1]
        self.match = int(row[len(row)-1])

In [2]:
def loadmatch(f,allnum=False):
    rows = []
    for line in file(f):
        rows.append(matchrow(line.split(','),allnum))
    return rows

In [3]:
agesonly = loadmatch('agesonly.csv')

In [ ]:
print agesonly[0].data,agesonly[0].match

In [4]:
matchmaker = loadmatch('matchmaker.csv')

In [ ]:
print matchmaker[0].data,matchmaker[0].match

In [5]:
from pylab import *

In [6]:
def plotagematches(rows):
    xdm,ydm = [r.data[0] for r in rows if r.match == 1],[r.data[1] for r in rows if r.match == 1]
    xdn,ydn = [r.data[0] for r in rows if r.match == 0],[r.data[1] for r in rows if r.match == 0]
    #plot(xdm,ydm,'go')
    #plot(xdn,ydn,'ro')
    plot(xdm,ydm,'bo')
    plot(xdn,ydn,'b+')
    show()

In [ ]:
plotagematches(agesonly)

In [7]:
def lineartrain(rows):
    averages = {}
    counts = {}
  
    for row in rows:
        # Get the class of this point
        cl = row.match

        averages.setdefault(cl,[0.0]*(len(row.data)))
        counts.setdefault(cl,0)

        # Add this point to the averages
        for i in range(len(row.data)):
            averages[cl][i] += float(row.data[i])

        # Keep track of how many points in each class
        counts[cl] += 1
    
    # Divide sums by counts to get the averages
    for cl,avg in averages.items():
        for i in range(len(avg)):
            avg[i] /= counts[cl]

    return averages

In [8]:
avgs = lineartrain(agesonly)

In [ ]:
avgs

In [9]:
def dotproduct(v1,v2):
    return sum([v1[i]*v2[i] for i in range(len(v1))])

def veclength(v):
    return sum([p**2 for p in v])

In [10]:
def dpclassify(point,avgs):
    b = (dotproduct(avgs[1],avgs[1]) - dotproduct(avgs[0],avgs[0]))/2
    y = dotproduct(point,avgs[0]) - dotproduct(point,avgs[1]) + b
    if y>0: return 0
    else: return 1

In [ ]:
dpclassify([30,30],avgs)

In [ ]:
dpclassify([25,30],avgs)

In [ ]:
dpclassify([35,30],avgs)

In [11]:
def yesno(v):
    if v=='yes': return 1
    elif v=='no': return -1
    else: return 0

In [12]:
def matchcount(interest1,interest2):
    l1 = interest1.split(':')
    l2 = interest2.split(':')
    x = 0
    for v in l1:
        if v in l2: x += 1
    return x

# Yahoo

In [13]:
from xml.dom.minidom import parseString
from urllib import urlopen,quote_plus

In [14]:
yahookey="YOUR API KEY"

In [15]:
loc_cache = {}

In [16]:
def getlocation(address):
    if address in loc_cache: return loc_cache[address]
    data = urlopen('http://api.local.yahoo.com/MapsService/V1/' + \
                    'geocode?appid=%s&location=%s' % (yahookey,quote_plus(address))).read()
    doc = parseString(data)
    lat = doc.getElementsByTagName('Latitude')[0].firstChild.nodeValue
    long = doc.getElementsByTagName('Longitude')[0].firstChild.nodeValue  
    loc_cache[address] = (float(lat),float(long))
    return loc_cache[address]

In [17]:
def milesdistance(a1,a2):
    return 0
    lat1,long1 = getlocation(a1)
    lat2,long2 = getlocation(a2)
    latdif = 69.1 * (lat2 - lat1)
    longdif = 53.0 * (long2 - long1)
    return (latdif**2 + longdif**2) ** 0.5

In [18]:
def loadnumerical():
    oldrows = loadmatch('matchmaker.csv')
    newrows = []
    for row in oldrows:
        d = row.data
        data = [float(d[0]),yesno(d[1]),yesno(d[2]),
                float(d[5]),yesno(d[6]),yesno(d[7]),
                matchcount(d[3],d[8]),
                milesdistance(d[4],d[9]),
                row.match]
        newrows.append(matchrow(data))
    return newrows

In [19]:
numericalset = loadnumerical()

In [ ]:
numericalset[0].data

In [ ]:
numericalset[0].match

In [20]:
def scaledata(rows):
    low = [999999999.0] * len(rows[0].data)
    high = [-999999999.0] * len(rows[0].data)
    # Find the lowest and highest values
    for row in rows:
        d = row.data
        for i in range(len(d)):
            if d[i] < low[i]: low[i] = d[i]
            if d[i] > high[i]: high[i] = d[i]
  
    # Create a function that scales data
    def scaleinput(d):
        return [(d[i]-low[i])/(high[i]-low[i] + 0.001) for i in range(len(low))]
  
    # Scale all the data
    newrows = [matchrow(scaleinput(row.data) + [row.match]) for row in rows]
  
    # Return the new data and the function
    return newrows,scaleinput

In [21]:
scaleset,scalef = scaledata(numericalset)

In [ ]:
scaleset[0].data

In [ ]:
scaleset[0].match

In [22]:
avgs = lineartrain(scaleset)

In [ ]:
avgs

In [ ]:
dpclassify(scalef(numericalset[0].data),avgs)

In [ ]:
numericalset[0].match

In [ ]:
numericalset[11].data

In [ ]:
numericalset[21].match

In [ ]:
dpclassify(scalef(numericalset[11].data),avgs)

In [ ]:
dpclassify(scalef(numericalset[21].data),avgs)

In [23]:
def rbf(v1,v2,gamma=10):
    dv = [v1[i]-v2[i] for i in range(len(v1))]
    # veclength = sum(every item pow pos)  veclength[1,2,3] = 1 + pow(2,2) + pow(3,3)
    l = veclength(dv)
    return math.e**(-gamma*l)

In [ ]:
 l = veclength([1,2,3])

In [ ]:
l

In [24]:
def nlclassify(point,rows,offset,gamma=10):
    sum0 = 0.0
    sum1 = 0.0
    count0 = 0
    count1 = 0
  
    for row in rows:
        if row.match == 0:
            sum0 += rbf(point,row.data,gamma)
            count0 += 1
        else:
            sum1 += rbf(point,row.data,gamma)
            count1 += 1
            
    y = (1.0/count0)*sum0 - (1.0/count1)*sum1 + offset

    if y > 0: return 0
    else: return 1

In [25]:
def getoffset(rows,gamma=10):
    l0 = []
    l1 = []
    for row in rows:
        if row.match == 0: l0.append(row.data)
        else: l1.append(row.data)
    
    sum0 = sum(sum([rbf(v1,v2,gamma) for v1 in l0]) for v2 in l0)
    sum1 = sum(sum([rbf(v1,v2,gamma) for v1 in l1]) for v2 in l1)

    return (1.0/(len(l1)**2))*sum1 - (1.0/(len(l0)**2))*sum0

In [26]:
ssoffset = getoffset(scaleset)
ssoffset

0.014938457177104847

0.014938457177104847

In [ ]:
nlclassify(scalef(numericalset[0].data),scaleset,ssoffset)

In [ ]:
nlclassify(scalef(numericalset[2].data),scaleset,ssoffset)

In [ ]:
numericalset[2].match

In [ ]:
newrow = [28.0,-1,-1,26.0,-1,1,2,0]

In [ ]:
nlclassify(scalef(newrow),scaleset,ssoffset)

In [ ]:
newrow = [28.0,-1,1,26.0,-1,1,2,0]

In [ ]:
nlclassify(scalef(newrow),scaleset,ssoffset)

# SVM

In [ ]:
"""
...
'options':
    -s svm_type : set type of SVM (default 0)
        0 -- C-SVC
        1 -- nu-SVC
        2 -- one-class SVM
        3 -- epsilon-SVR
        4 -- nu-SVR
    -t kernel_type : set type of kernel function (default 2)
        0 -- linear: u'*v
        1 -- polynomial: (gamma*u'*v + coef0)^degree
        2 -- radial basis function: exp(-gamma*|u-v|^2)
        3 -- sigmoid: tanh(gamma*u'*v + coef0)
        4 -- precomputed kernel (kernel values in training_set_file)
    -d degree : set degree in kernel function (default 3)
    -g gamma : set gamma in kernel function (default 1/num_features)
    -r coef0 : set coef0 in kernel function (default 0)
    -c cost : set the parameter C of C-SVC, epsilon-SVR, and nu-SVR (default 1)
    -n nu : set the parameter nu of nu-SVC, one-class SVM, and nu-SVR (default 0.5)
    -p epsilon : set the epsilon in loss function of epsilon-SVR (default 0.1)
    -m cachesize : set cache memory size in MB (default 100)
    -e epsilon : set tolerance of termination criterion (default 0.001)
    -h shrinking : whether to use the shrinking heuristics, 0 or 1 (default 1)
    -b probability_estimates : whether to train a SVC or SVR model for probability estimates, 0 or 1 (default 0)
    -wi weight : set the parameter C of class i to weight*C, for C-SVC (default 1)
    -v n: n-fold cross validation mode
    -q : quiet mode (no outputs)
"""

In [27]:
from svm import *
from svmutil import *

In [28]:
prob = svm_problem([1 , -1] , [[1,0, 1] , [ -1 ,0, -1] ])

In [29]:
m = svm_train(prob, '-t 0 -c 10')

In [30]:
m = svm_train([1,-1],[[1,0,1],[-1,0,-1]], '-t 0 -c 10')
p_labels, p_acc, p_vals = svm_predict([1,-1],[[1,0,1],[-1,0,-1]], m)

Accuracy = 100% (2/2) (classification)
Accuracy = 100% (2/2) (classification)


In [ ]:
p_labels

In [ ]:
p_acc

In [ ]:
p_vals

In [36]:
p_labels, p_acc, p_vals = svm_predict([-1],[[1,1,1]], m)
print p_labels,p_acc,p_vals

Accuracy = 0% (0/1) (classification)
[1.0] (0.0, 4.0, nan) [[1.0]]
Accuracy = 0% (0/1) (classification)
[1.0] (0.0, 4.0, nan) [[1.0]]


In [37]:
from svmutil import *
# Specify training set
prob = svm_problem([1,-1],[[1,0,1],[-1,0,-1]])
# Train the model
m = svm_train(prob, '-t 0 -c 1')
# Make a prediction
predicted_labels, _, _ = svm_predict([-1],[[1,1,1]],m)
# Predicted label for input [1,1,1] is predicted_labels[0]
print "Predicted value: " + str(predicted_labels[0])

Accuracy = 0% (0/1) (classification)
Predicted value: 1.0
Accuracy = 0% (0/1) (classification)
Predicted value: 1.0


In [38]:
answers,inputs = [r.match for r in scaleset],[r.data for r in scaleset]

In [41]:
m = svm_train(svm_problem(answers,inputs),'-t 2 -c 1')

In [42]:
newrow = [28.0,-1,-1,26.0,-1,1,2,0]

In [44]:
predicted_labels, _, _ = svm_predict([1],[scalef(newrow)],m)

Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)


In [45]:
predicted_labels

[0.0]

[0.0]

In [47]:
newrow = [28.0,-1,1,26.0,-1,1,2,0]

In [48]:
predicted_labels, _, _ = svm_predict([1],[scalef(newrow)],m)

Accuracy = 100% (1/1) (classification)
Accuracy = 100% (1/1) (classification)


In [49]:
predicted_labels

[1.0]

[1.0]

In [53]:
guesses = cross_validation(svm_problem(answers,inputs),'-t 2 -c 1',4,)

NameError: name 'cross_validation' is not defined

NameError: name 'cross_validation' is not defined

# Facebook

In [54]:
import urllib,md5,webbrowser,time
from xml.dom.minidom import parseString

C:\Python\Python27\lib\site-packages\ipykernel-4.1.1-py2.7.egg\ipykernel\__main__.py:1: DeprecationWarning: the md5 module is deprecated; use hashlib instead
  if __name__ == '__main__':
C:\Python\Python27\lib\site-packages\ipykernel-4.1.1-py2.7.egg\ipykernel\__main__.py:1: DeprecationWarning: the md5 module is deprecated; use hashlib instead
  if __name__ == '__main__':


In [58]:
apikey = "47e953c8ea9ed30db904af453125c759"
secret = "ea703e4721e8c7bf88b92110a46a9b06"
FacebookURL = "https://api.facebook.com/restserver.php"

In [56]:
def getsinglevalue(node,tag):
    nl=node.getElementsByTagName(tag)
    if len(nl)>0:
        tagNode=nl[0]
        if tagNode.hasChildNodes():
            return tagNode.firstChild.nodeValue
    return ''

In [57]:
def callid(): 
    return str(int(time.time()*10))

In [60]:
class fbsession:
    def __init__(self):
        self.session_secret=None
        self.session_key=None
        self.createtoken()
        webbrowser.open(self.getlogin())
        print "Press enter after logging in:",
        raw_input()
        self.getsession()
        
    def sendrequest(self, args):
        args['api_key'] = apikey
        args['sig'] = self.makehash(args)
        post_data = urllib.urlencode(args)
        url = FacebookURL + "?" + post_data
        data=urllib.urlopen(url).read()
        print data
        return parseString(data)
    
    def makehash(self,args):
        hasher = md5.new(''.join([x + '=' + args[x] for x in sorted(args.keys())]))
        if self.session_secret: hasher.update(self.session_secret)
        else: hasher.update(secret)
        return hasher.hexdigest()    
    
    def createtoken(self):
        res = self.sendrequest({'method':"facebook.auth.createToken"})
        self.token = getsinglevalue(res,'token')
        
    def getlogin(self):
        return "http://api.facebook.com/login.php?api_key=" + apikey + "&auth_token=" + self.token
    
    def getsession(self):
        doc = self.sendrequest({'method':'facebook.auth.getSession','auth_token':self.token})
        self.session_key = getsinglevalue(doc,'session_key')
        self.session_secret = getsinglevalue(doc,'secret')
    
    def getfriends(self):
        doc = self.sendrequest({'method':'facebook.friends.get','session_key':self.session_key,'call_id':callid()})
        results = []
        for n in doc.getElementsByTagName('result_elt'):
            results.append(n.firstChild.nodeValue)
        return results  
    
    def getinfo(self,users):
        ulist = ','.join(users)

        fields = 'gender,current_location,relationship_status,affiliations,hometown_location'

        doc = self.sendrequest({'method':'facebook.users.getInfo',
                                'session_key':self.session_key,'call_id':callid(),
                                'users':ulist,'fields':fields})

        results={}
        for n,id in zip(doc.getElementsByTagName('result_elt'),users):
            # Get the location
            locnode = n.getElementsByTagName('hometown_location')[0]
            loc = getsinglevalue(locnode,'city') + ', ' + getsinglevalue(locnode,'state')

            # Get school
            college = ''
            gradyear = '0'
            affiliations = n.getElementsByTagName('affiliations_elt')
            for aff in affiliations:
                # Type 1 is college
                if getsinglevalue(aff,'type') == '1': 
                    college = getsinglevalue(aff,'name')
                    gradyear = getsinglevalue(aff,'year')

                results[id] = {'gender':getsinglevalue(n,'gender'),
                               'status':getsinglevalue(n,'relationship_status'),
                               'location':loc,'college':college,'year':gradyear}
        return results  
    
    def arefriends(self,idlist1,idlist2):
        id1 = ','.join(idlist1)
        id2 = ','.join(idlist2)
        doc = self.sendrequest({'method':'facebook.friends.areFriends',
                                'session_key':self.session_key,'call_id':callid(),
                                'id1':id1,'id2':id2})
        results=[]
        for n in doc.getElementsByTagName('result_elt'):
            results.append(int(n.firstChild.nodeValue))
        return results    

    def makedataset(self):
        from advancedclassify import milesdistance
        # Get all the info for all my friends
        friends = self.getfriends()
        info = self.getinfo(friends)
        ids1,ids2 = [],[]
        rows=[]

        # Nested loop to look at every pair of friends
        for i in range(len(friends)):
            f1 = friends[i]
            data1 = info[f1]

            # Start at i+1 so we don't double up
            for j in range(i+1,len(friends)):
                f2 = friends[j]
                data2 = info[f2]
                ids1.append(f1)
                ids2.append(f2)

                # Generate some numbers from the data
                if data1['college'] == data2['college']: sameschool=1
                else: sameschool=0
                    
                male1 = (data1['gender']=='Male') and 1 or 0
                male2 = (data2['gender']=='Male') and 1 or 0        

                row = [male1,int(data1['year']),male2,int(data2['year']),sameschool]
                rows.append(row)
        # Call arefriends in blocks for every pair of people
        arefriends = []
        for i in range(0,len(ids1),30):
            j = min(i+30,len(ids1))
            pa = self.arefriends(ids1[i:j],ids2[i:j])
            arefriends += pa
        return arefriends,rows
  
    

In [61]:
s = fbsession()
answers,data = s.makedataset()


IOError: [Errno socket error] [Errno 10061] 

IOError: [Errno socket error] [Errno 10061] 

In [62]:
m = svm_train(svm_problem(answers,data),'-t 2 -c 1')
newrow = [1,2003,1,2003,1]
predicted_labels, _, _ = svm_predict([1],[newrow],m)
newrow = [1,2003,1,1996,0]
predicted_labels, _, _ = svm_predict([0],[newrow],m)

NameError: name 'data' is not defined

NameError: name 'data' is not defined